In [2]:
import collections
import math
import os
import random
import tarfile
import re
from six.moves import urllib
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
import tensorflow as tf

/Users/i550012/virtualenv/tensorflow1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/i550012/virtualenv/tensorflow1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/i550012/virtualenv/tensorflow1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/i550012/virtualenv/tensorflow1

In [3]:
DOWNLOADED_FILENAME = 'ImdbReviews.tar.gz'
def download_file(url_path):
    if not os.path.exists(DOWNLOADED_FILENAME):
        filename, _ = urllib.request.urlretrieve(url_path + DOWNLOADED_FILENAME, DOWNLOADED_FILENAME)
    print('Found and verified file from this path: ', url_path + DOWNLOADED_FILENAME)
    print('Downloaded file: ', DOWNLOADED_FILENAME)

In [9]:
TOKEN_REGEX = re.compile("[^A-Za-z0-9 ]+")
def get_reviews(dirname, positive=True):
    label = 1 if positive else 0
    reviews = []
    labels = []
    for file in os.listdir(dirname):
        if file.endswith(".txt"):
            with open(dirname + file, 'r+') as f:
                review = f.read()
                review = review.lower().replace("<br />", " ")
                review = re.sub(TOKEN_REGEX, '', review)
                reviews.append(review)
                labels.append(label)
    return reviews, labels

     
    

In [5]:
def extract_labels_data():
    if not os.path.exists('aclImdb'):
        with tarfile.open(DOWNLOADED_FILENAME) as tar:
            tar.extractall()
            tar.close()
    positive_reviews, positive_labels = get_reviews("aclImdb/train/pos/", positive=True)
    negative_reviews, negative_labels = get_reviews("aclImdb/train/neg/",positive=False)
    data = positive_reviews+negative_reviews
    labels = positive_labels+negative_labels
    return data, labels

In [7]:
URL_PATH = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
download_file(URL_PATH)


Found and verified file from this path:  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gzImdbReviews.tar.gz
Downloaded file:  ImdbReviews.tar.gz


In [10]:
data,labels = extract_labels_data()

In [11]:
len(labels),len(data)

(25000, 25000)

In [12]:
max_document_length = max([len(x.split(" ")) for x in data])
print(max_document_length)

2470


In [13]:
MAX_SEQUENCE_LENGTH = 250

In [14]:
vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(MAX_SEQUENCE_LENGTH)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.


In [15]:
x_data = np.array(list(vocab_processor.fit_transform(data)))
y_output = np.array(labels)
vocabulary_size = len(vocab_processor.vocabulary_)
print(vocabulary_size)

Instructions for updating:
Please use tensorflow/transform or tf.data.
111526


In [16]:
np.random.seed(42)
shuffle_indices = np.random.permutation(np.arange(len(x_data)))
x_shuffled = x_data[shuffle_indices]
y_shuffled = y_output[shuffle_indices]

In [17]:
TRAIN_DATA = 5000
TOTAL_DATA = 6000

train_data = x_shuffled[:TRAIN_DATA]
train_target = y_shuffled[:TRAIN_DATA]

test_data = x_shuffled[TRAIN_DATA:TOTAL_DATA]
test_target = y_shuffled[TRAIN_DATA:TOTAL_DATA]


In [18]:
tf.reset_default_graph()

x = tf.placeholder(tf.int32, [None, MAX_SEQUENCE_LENGTH])
y = tf.placeholder(tf.int32,[None])


In [19]:
num_epochs = 20
batch_size = 25
embedding_size = 50
max_label = 2

In [20]:
embedding_matrix = tf.Variable(tf.random_uniform([vocabulary_size,embedding_size],-1.0,1.0))
embeddings = tf.nn.embedding_lookup(embedding_matrix,x)


In [21]:
embedding_matrix.get_shape(),embeddings.get_shape()

(TensorShape([Dimension(111526), Dimension(50)]),
 TensorShape([Dimension(None), Dimension(250), Dimension(50)]))

In [22]:
lstmCell = tf.contrib.rnn.BasicLSTMCell(embedding_size)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell,output_keep_prob=0.75)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


In [23]:
_,(encoding,_)=tf.nn.dynamic_rnn(lstmCell,embeddings,dtype=tf.float32)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [24]:
encoding.get_shape()

TensorShape([Dimension(None), Dimension(50)])

In [25]:
logits = tf.layers.dense(encoding,max_label,activation=None)

Instructions for updating:
Use keras.layers.dense instead.


In [26]:
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=y)
loss = tf.reduce_mean(cross_entropy)


In [28]:
prediction = tf.equal(tf.argmax(logits, 1), tf.cast(y,tf.int64))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))


In [29]:
optimizer = tf.train.AdamOptimizer(0.01)
train_step = optimizer.minimize(loss)



In [30]:
init = tf.global_variables_initializer()

In [31]:
with tf.Session() as sess:
    init.run()
    for epoch in range(num_epochs):
        num_batches = int(len(train_data)//batch_size)+1
        for i in range(num_batches):
            min_ix = i*batch_size
            max_ix = np.min([len(train_data), ((i+1)*batch_size)])
            x_train_batch = train_data[min_ix:max_ix]
            y_train_batch = train_target[min_ix:max_ix]

            train_dict = {x: x_train_batch, y: y_train_batch}
            sess.run(train_step, feed_dict=train_dict)
            
            train_loss, train_acc = sess.run([loss, accuracy], feed_dict=train_dict)
        test_dict = {x: test_data, y: test_target}
        test_loss, test_acc = sess.run([loss, accuracy], feed_dict=test_dict)
        print('Epoch: {}, TestLoss: {:.2}, TestAcc: {:.5}'.format(epoch + 1, test_loss, test_acc))

2024-06-15 05:45:40.121680: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA


Epoch: 1, TestLoss: 0.69, TestAcc: 0.541
Epoch: 2, TestLoss: 0.81, TestAcc: 0.538
Epoch: 3, TestLoss: 1.0, TestAcc: 0.612
Epoch: 4, TestLoss: 0.8, TestAcc: 0.704
Epoch: 5, TestLoss: 1.0, TestAcc: 0.741
Epoch: 6, TestLoss: 1.1, TestAcc: 0.759
Epoch: 7, TestLoss: 1.4, TestAcc: 0.769
Epoch: 8, TestLoss: 1.5, TestAcc: 0.778
Epoch: 9, TestLoss: 1.7, TestAcc: 0.78
Epoch: 10, TestLoss: 1.7, TestAcc: 0.781
Epoch: 11, TestLoss: 1.8, TestAcc: 0.782
Epoch: 12, TestLoss: 1.9, TestAcc: 0.782
Epoch: 13, TestLoss: 1.9, TestAcc: 0.781
Epoch: 14, TestLoss: 2.0, TestAcc: 0.78
Epoch: 15, TestLoss: 2.0, TestAcc: 0.779
Epoch: 16, TestLoss: 2.1, TestAcc: 0.782
Epoch: 17, TestLoss: 2.1, TestAcc: 0.781
Epoch: 18, TestLoss: 2.2, TestAcc: 0.782
Epoch: 19, TestLoss: 2.3, TestAcc: 0.781
Epoch: 20, TestLoss: 2.3, TestAcc: 0.778
